In [1]:
!git clone https://github.com/ultralytics/yolov5  # clone
!cd yolov5
!pip install -r yolov5/requirements.txt
!pip install wandb==0.12.10 

fatal: destination path 'yolov5' already exists and is not an empty directory.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3.9 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3.9 -m pip install --upgrade pip' command.


In [2]:
%matplotlib inline
import pandas as pd
import numpy as np
import glob
from sklearn import cluster
# !pip install scikit-image
import io
import gc
# import progressbar
import random
from os.path import exists

import matplotlib.pyplot as plt
from matplotlib import patches

from tqdm import tqdm
from itertools import chain

import PIL
from pathos.pools import ProcessPool
from PIL import Image
import json
import os
from sklearn.model_selection import KFold

from engine import train_one_epoch, evaluate
import base64
import torch
import torchvision
import torchvision as tv
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
from torchvision import transforms as torchtrans
import albumentations as A
from termcolor import colored
!pip install imgaug
import imgaug
import copy
from coco_utils import get_coco_api_from_dataset
from coco_eval import CocoEvaluator
import wandb
# these are the helper libraries imported.

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3.9 -m pip install --upgrade pip' command.


In [3]:
# wandb.login()

In [4]:
num_aug = 16

In [5]:
train_files = glob.glob("/home/roger/train/*.json")

train_files = np.array(sorted(train_files, key = lambda x: int(x.split("/")[-1].split(".")[0].split("_")[-1])))

train_files = list(np.repeat(train_files, num_aug))

In [6]:
val_files = glob.glob("/home/roger/val/*.json")

val_files = list(sorted(val_files, key = lambda x: int(x.split("/")[-1].split(".")[0].split("_")[-1])))

In [7]:
def pascal2yolo(size, box):
    dw = 1./size[0]
    dh = 1./size[1]
    x = (box[0] + box[2])/2.0
    y = (box[1] + box[3])/2.0
    w = box[2] - box[0]
    h = box[3] - box[1]
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    return (x,y,w,h)

In [8]:
def processJson(file, idx, base_path, classes, width, height, max_size = 1080):
    
    img_name = f"{base_path}/images/train_{idx}.jpg"
    label_name = f"{base_path}/labels/train_{idx}.txt"
    
    if exists(img_name) or exists(label_name):
        print("Data already exists! Skipping...")
        return
    
    f = open(file)
    data = json.load(f)

    img = io.BytesIO()
    img.write(base64.b64decode(data["imageData"]))
    x = np.array(PIL.Image.open(img))
    y = {}
    
    boxes = [list(np.sort(np.array(box["points"]), axis = 0).flatten()) for box in data["shapes"] if box["label"] == "void"]
    labels = [classes.index(box["label"]) for box in data["shapes"] if box["label"] == "void"]

    shift_x = np.linspace(0,80,10)
    shift_y = np.linspace(0,80,10)
    cont = [-0.2,0.2]
    bright = [-0.2, 0.2]

    M=0
    p=0.4

    teacher_transform = A.Compose(
        [
            A.augmentations.geometric.resize.LongestMaxSize(max_size = 1080),
            A.RandomCrop(width=width, height=height, p=1),
            A.ShiftScaleRotate(shift_limit_x=shift_x[M], rotate_limit=5, shift_limit_y=shift_y[M], p=p),

            A.RandomBrightnessContrast(brightness_limit = bright, contrast_limit=cont, p=p),
            A.augmentations.transforms.Flip(p=p),
        ], bbox_params=A.BboxParams(format='pascal_voc', label_fields = ["class_labels"], min_visibility = 0.5)
    )

    transformed = teacher_transform(image=x, bboxes=boxes, class_labels = labels)


    aug_x = transformed['image']
    boxes = np.reshape(transformed['bboxes'], (-1,4))
    
    classes = transformed['class_labels']

    im = Image.fromarray(aug_x)
    im.save(img_name)

    with open(label_name, 'w') as f:
        for idx, box in enumerate(boxes):
            bc = pascal2yolo((width, height), box)
            c = classes[idx]
            f.write(f"{c} {bc[0]} {bc[1]} {bc[2]} {bc[3]}\n")


In [9]:
def generate_data(files, output):
    for idx, file in enumerate(tqdm(files)):
        random.seed(idx)
        processJson(file, idx, output, ["void"], 576, 576, max_size = 1080)

In [10]:
generate_data(train_files, "/home/roger/drone/Empty-Shelves/train")

100%|██████████████████████████████████████████████████████████████████████| 2880/2880 [00:00<00:00, 48653.90it/s]

Data already exists! Skipping...
Data already exists! Skipping...
Data already exists! Skipping...
Data already exists! Skipping...
Data already exists! Skipping...
Data already exists! Skipping...
Data already exists! Skipping...
Data already exists! Skipping...
Data already exists! Skipping...
Data already exists! Skipping...
Data already exists! Skipping...
Data already exists! Skipping...
Data already exists! Skipping...
Data already exists! Skipping...
Data already exists! Skipping...
Data already exists! Skipping...
Data already exists! Skipping...
Data already exists! Skipping...
Data already exists! Skipping...
Data already exists! Skipping...
Data already exists! Skipping...
Data already exists! Skipping...
Data already exists! Skipping...
Data already exists! Skipping...
Data already exists! Skipping...
Data already exists! Skipping...
Data already exists! Skipping...
Data already exists! Skipping...
Data already exists! Skipping...
Data already exists! Skipping...
Data alrea

In [11]:
generate_data(val_files, "/home/roger/drone/Empty-Shelves/val")

100%|██████████████████████████████████████████████████████████████████████████| 38/38 [00:00<00:00, 27385.49it/s]

Data already exists! Skipping...
Data already exists! Skipping...
Data already exists! Skipping...
Data already exists! Skipping...
Data already exists! Skipping...
Data already exists! Skipping...
Data already exists! Skipping...
Data already exists! Skipping...
Data already exists! Skipping...
Data already exists! Skipping...
Data already exists! Skipping...
Data already exists! Skipping...
Data already exists! Skipping...
Data already exists! Skipping...
Data already exists! Skipping...
Data already exists! Skipping...
Data already exists! Skipping...
Data already exists! Skipping...
Data already exists! Skipping...
Data already exists! Skipping...
Data already exists! Skipping...
Data already exists! Skipping...
Data already exists! Skipping...
Data already exists! Skipping...
Data already exists! Skipping...
Data already exists! Skipping...
Data already exists! Skipping...
Data already exists! Skipping...
Data already exists! Skipping...
Data already exists! Skipping...
Data alrea

In [ ]:
!python3.9 yolov5/train.py --img 576 --batch 1 --epochs 300 --data empty_shelves.yaml --weights yolov5x.pt

wandb: Currently logged in as: goggins (use `wandb login --relogin` to force relogin)
train: weights=yolov5x.pt, cfg=, data=empty_shelves.yaml, hyp=yolov5/data/hyps/hyp.scratch-low.yaml, epochs=300, batch_size=1, imgsz=576, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=yolov5/runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
/home/roger/.local/lib/python3.9/site-packages/torch/cuda/__init__.py:122: UserWarning: 
    Found GPU1 NVS 510 which is of cuda capability 3.0.
    PyTorch no longer supports this GPU because it is too old.
    The minimum cuda capability su